The first cell in your notebook is the doc string. This is used by the <b>docgen</b> package to create the README.md file in the main project repository. See example below.

In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Corporate Actions in LUSID

Attributes
----------
corporate actions
transactions
derived portfolios
holdings
"""

toggle_code("Toggle Docstring")

# FINBOURNE notebook style guide

The title should be in markdown H1 followed by a short paragraph summarising the notebook contents. 

Example:

This notebook outlines the Jupyter style guide for FINBOURNE. 

The summary can also include a table of contents:


###  Table of contents

* [Setup](#setup)
* [Code style guide](#code-style)
* [Markdown and writing style guide](#writing-style)

## Setup

The setup section should contain 3 code cells:

* The first code cell in the setup is used to import packages and connect to LUSID.

In [2]:
# Use first block to import generic non-LUSID packages
import os
import pandas as pd
import datetime
import json
from IPython.core.display import HTML

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "${:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.5588.0,0.5.1720,"{'relation': 'RequestLogs', 'href': 'http://st..."


* The second code cell in the setup is used to declare the api we'll use throughout the notebook.

In [3]:
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
scopes_api = api_factory.build(la.ScopesApi)
instruments_api = api_factory.build(la.InstrumentsApi)

* The third code cell in the setup is used to define variables which are re-used throughout the notebook.

In [4]:
portfolio_code = "equityTech"
scope = "emeaIBOR"
created_date = "2020-01-01"

## Notebook code style guide <a name = "code-style"></a>

This section outlines code style and structure recommendations.

### 1) Run <i>black</i> on all Python code to ensure consistent code formatting

Use the [black](https://github.com/psf/black) package to ensure clean and consistent code.

Note - this command should not be committed to the published version of your notebook. Rather you should run the command before commiting to ensure that other code cells are formatted nicely.

In [5]:
# Uncomment this magic command to run
# %load_ext nb_black

### 2) camelCase should be used for all variable values

We use camelCase for variable values. 

This keeps our code consistent with the case of the underlying API parameters.

Example: <b>lusidInstrumentId</b>

See the [docs](https://www.lusid.com/docs/api/) for more details.

In [6]:
portfolio_code = "equityTech"

### 3) However snake_case is used for variable names

This keeps the format consistent with Python's [PEP 8](https://www.python.org/dev/peps/pep-0008/#function-and-variable-names)

In [7]:
portfolio_location = "emeaRegion"

### 4) You should Create rather than Set new Transaction Type configurations

We use [CreateConfigurationTransactionType](https://www.lusid.com/docs/api/#operation/CreateConfigurationTransactionType) rather than [SetConfigurationTransactionTypes](https://www.lusid.com/docs/api/#operation/SetConfigurationTransactionTypes) when creating new `Transaction Types` within a notebook. 

Why?
The Create method appends your new transaction type configuration to the current set whereas the Set method overrides the current set. `Transaction Types` are system wide settings which drive much of LUSID's movement engine functionality and modifying them can have unintended consequences.


### 5) All notebooks should be idempotent

Your notebook should be able to run (a) across any LUSID environment; and (b) without depancies on data setup which happens outside the notebook. 

For example, consider a notebook function which needs prices for an instrument. Those prices should be set within the same notebook so that a new user with a new environment can run the notebook successfully.

### 6) Handle exceptions gracefully with sensible try except blocks

You should use LUSID's [error codes](https://www.lusid.com/docs/api/#section/Error-Codes) to handle exceptions in your code. This will also help keep the notebooks idempotent. Consider the example below. The notebook would not be idempotent if you did not handle the exception - it would run successfully first run, but fail for each subsequent run.

In [8]:
# Define a portfolio and scope for our request
portfolio_scope = "equityPortfolios"
portfolio_code = "techPortfolio"


# Try and get a portfolio called techPortfolio
# If that portfolio does not exists, then create it

try:

    tech_portfolio = portfolios_api.get_portfolio(
        scope=portfolio_scope, code=portfolio_code
    )

    display(f"Found portfolio {portfolio_code} in scope {portfolio_scope}")

except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioNotFound":
        create_portfolio = transaction_portfolios_api.create_portfolio(
            scope=portfolio_scope,
            create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
                code=portfolio_code, display_name=portfolio_code, base_currency="USD"
            ),
        )

    display(f"Created new portfolio {portfolio_code} in scope {portfolio_scope}")

'Found portfolio techPortfolio in scope equityPortfolios'

### 7) Avoid using GUIDs for scopes and codes (if possible)

You should avoid creating a new GUID for scopes and codes if possible. These notebooks run everyday in our CI environment and its costly to create a new scope and/or code for each run. Instead you should try and upsert a static code and scope. To ensure idempotency, you can then:

1. Use `upsert` methods where possible

These can be used to reset your data. For example, you can `upsert` a set of transactions at the start of your notebook. The upsert should bring the transactions back into a desired state if they had already been modified by a previous run of the notebook.  

2. Otherwise implement a try except block (as above)

You can also implement a try except block. Consider the example above.

3. Tear down any changes at the end of your notebook

Finally you can tear-down any changes at the end of your notebook. You can cancel or reverse any changes made throughout the notebook, so it starts from it's original state during the next run.

### 8) Datetimes should be declared using datetime objects

Dates and times should be declared using the datetime object rather than a string.

In [9]:
start_date = datetime.datetime(year=2020, month=6, day=20)

And date strings should be printed in [ISO](https://en.wikipedia.org/wiki/ISO_8601) format:

In [10]:
start_date.strftime(format="%Y-%m-%d")

'2020-06-20'

### 9) Trigger a notebook to stop

The following exception and format should be used to stop a notebook running (if required).

In this example, we are stopping the notebook due to a missing portfolio.

In [11]:
# Change conditional from next line to raise the exception

if False:  # Change to True to stop the notebook
    raise StopExecution(f"Missing portfolio {portfolio_code}")
else:
    pass

### 10) Convert LUSID responses to DataFrames rather than showing raw JSON

We love JSON at FINBOURNE but DataFrames look better for notebooks:

In [12]:
# The lusid_response_to_data_frame function converts the LUSID JSON response to a DataFrame

instruments_df = lusid_response_to_data_frame(
    instruments_api.list_instruments(limit=3), rename_properties=True
)

display(instruments_df)

,href,lusid_instrument_id,version.effective_from,version.as_at_date,name,identifiers.Currency,identifiers.LusidInstrumentId,properties,state
0,https://stephenlm.lusid.com/api/api/instrument...,CCY_AED,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,AED,AED,CCY_AED,[],Active
1,https://stephenlm.lusid.com/api/api/instrument...,CCY_AFN,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,AFN,AFN,CCY_AFN,[],Active
2,https://stephenlm.lusid.com/api/api/instrument...,CCY_ALL,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,ALL,ALL,CCY_ALL,[],Active


You can also change the default column names using the <b>column_name_mapping</b> parameter of <b>lusid_response_to_data_frame</b>.

Example:

In [13]:
# Define dict of old name: new name

column_name_mapping = {
    "lusid_instrument_id": "LUID",
    "identifiers.Currency": "InstrumentCcy",
}

# Pass the new column_name_mapping parameter to the function
instruments_df = lusid_response_to_data_frame(
    instruments_api.list_instruments(limit=3),
    column_name_mapping=column_name_mapping,
    rename_properties=True,
)

display(instruments_df)

,href,LUID,version.effective_from,version.as_at_date,name,InstrumentCcy,identifiers.LusidInstrumentId,properties,state
0,https://stephenlm.lusid.com/api/api/instrument...,CCY_AED,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,AED,AED,CCY_AED,[],Active
1,https://stephenlm.lusid.com/api/api/instrument...,CCY_AFN,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,AFN,AFN,CCY_AFN,[],Active
2,https://stephenlm.lusid.com/api/api/instrument...,CCY_ALL,0001-01-01 00:00:00+00:00,2020-10-21 11:32:45.741765+00:00,ALL,ALL,CCY_ALL,[],Active


### 11) Use display() rather than print()

The display() function has a prettier output than print() in notebooks. Also, display will show all items in a loop, and not just the last item.

In [14]:
%%capture 
# Use the capture magic command to surpress output for this notebook
# Note that we need to break Python comment convention here
# Comments should typically be placed above or beside code
# The %%capture command however needs to be places on line 1 in a Cell

print(instruments_df);
display(instruments_df);

### 12) The output of code cells should be committed to source control

These notebooks are designed to demonstrate functionality and use-cases of LUSID. They can be read and understood without actually running the code. Therefore you should include the output of your code in the published version. 

### 13) You should only import packages used in the notebook

Do not import packages which are not used in the notebook.

## Markdown and writing style guide <a name = "writing-style"></a>

This section outlines markdown and writing style recommendations.

### 1) Make use of headers (this is a H3 header)

The headers should be used as follows:

# H1 for notebook Title
## H2 for new sections
### H3 for sub sections

### 2) References to core LUSID data entities in the markdown should be highlighted 

For example, `Transactions`, `Instruments` and `Derived Portfolios` are all first class LUSID data entities. These should be highlighted so that the reader knows we are referring to a specific LUSID object rather than a more general concept.

### 3) Readers should be linked back to the Knowledge Base for core concepts

For example, in the sentence below we link the reader to the knowlege base to learn more about scopes.

Example:

"Add your portfolio into the [scope](https://support.finbourne.com/what-is-a-scope-in-lusid-and-how-is-it-used) called uKScope"

### 4) We use the Google tech writing rules as a guide

You can see the full documentation here:

[Tech writing](https://developers.google.com/tech-writing)

Some key points to highlight:

* Use terms consistently (example: don't jump between describing "positions" and "holdings")
* Avoid ambiguous pronouns
* Prefer active voice to passive voice
* Convert some long sentences to lists
* Eliminate unneeded words
* Use a numbered list when ordering is important and a bulleted list when ordering is irrelevant
* Focus each paragraph on a single topic.

Exceptions to this guide:

* We use British English rather than American English (Example: Colour and not Color)

### 5) Use markdown cells for commentary, only use # for code comments

Example: 
    
In LUSID, scopes are effectively "namespaces" for partitioning data.

In [15]:
# Return the list of scopes

scopes_response = scopes_api.list_scopes()

### 6) Don't leave an empty cell at the end of the notebook

A common way to run notebooks is to select "Ctrl+Enter" as you progress through the cells. This is a really useful shortcut however running "Ctrl+Enter"on the last cell will create a new empty cell. It's good practise to remove this blank cell before commiting to source control. 